In [1]:
from sqlalchemy import create_engine
from dotenv import load_dotenv
from zipfile import ZipFile
from io import StringIO
import pandas as pd
import requests
import os

Read the database connection details from the repository's `secrets` and create the `engine` that will be used to connect to the database.

In [16]:
load_dotenv('../.env')
host = os.getenv("DB_HOST")
user = os.getenv("DB_USERNAME")
passwd = os.getenv("DB_PASSWORD")
db = os.getenv("DB_NAME")

engine = create_engine(
    f'mysql+mysqlconnector://{user}:{passwd}@{host}/{db}?autocommit=true',
    echo=False,
    connect_args={'ssl_ca': '/etc/ssl/certs/ca-certificates.crt'},
    )

Download the zip file that contains the data from the CDC's website to the `/data/` directory.

In [3]:
if not os.path.exists('../data/LLCP2019ASC.zip'):
    url = 'https://www.cdc.gov/brfss/annual_data/2019/files/LLCP2019ASC.zip'
    response = requests.get(url, stream=True)
    with open('../data/LLCP2019ASC.zip', 'wb') as f:
        for chunk in response.iter_content(chunk_size=512):
            if chunk: 
                f.write(chunk)
    # Delete the `response` variable to avoid memory issues.
    del response

Read the zip file and extract each line of the file into a list of strings to later convert it to a dataframe.

In [4]:
data_rows = []
asc_zip = ZipFile('../data/LLCP2019ASC.zip', 'r')
file_in_zip = asc_zip.namelist()[0]
for line in asc_zip.open(file_in_zip).readlines():
    data_rows.append(line.decode('utf-8'))
asc_zip.close()

Delete the `asc_zip` and `file_in_zip` variables to avoid memory issues.

In [5]:
del asc_zip
del file_in_zip

Convert the list of strings into a dataframe and show the head and tail of the dataframe to understand its structure.

In [6]:
data = pd.DataFrame(data_rows,columns=['raw'])
data

,raw
0,01 0101182019 11002019000001 ...
1,01 0101132019 11002019000002 ...
2,01 0101182019 11002019000003 ...
3,01 0101182019 12002019000004 ...
4,01 0101042019 11002019000005 ...
...,...
418263,72 0903152020 11002019006029 ...
418264,72 0903082020 11002019006030 ...
418265,72 0903102020 11002019006031 ...
418266,72 0903062020 11002019006032 ...


Once we've seen how the data would be structured in a dataframe, delete the `data` variable to avoid memory issues.

In [7]:
del(data)

#### Retreive the metadata from the MySQL database

Query the MySQL database to retrieve the layout table, which describes the structure of the columns. 
The layout will be used to perform an action similar to the MS Excel action "Text to Columns", the raw data (strings) in the dataframe will be split into multiple columns.

_Note:_ An `if` condition is used to check if the query has already been executed and saved as a `.json` file, if so, the json file will be read instead. This is done to avoid exhausting the amount of free queries per month provided by the MySQL database provider. 

In [8]:
if not os.path.exists('../data/layout.json'):
    df = pd.io.sql.read_sql('SELECT * FROM layout', engine,index_col='Code')
    df.to_json('../data/layout.json',orient='table')

layout = pd.read_json('../data/layout.json', orient='table')
layout.head()

,Start,Length
Code,,
_STATE,1,2
FMONTH,17,2
IDATE,19,8
IMONTH,19,2
IDAY,21,2


Query the MySQL database to retrieve the `catalog` table. The `Value` column in this table can be used to find which columns contain `float` values. This information will be used to convert the columns to the correct data type.

In [9]:
if not os.path.exists('../data/catalog.json'):
    df = pd.io.sql.read_sql('SELECT * FROM catalog', engine,index_col=['Code','Value'])
    df.to_json('../data/catalog.json',orient='table')

catalog = pd.read_json('../data/catalog.json', orient='table')
catalog.head()

Value_Label  Frequency  Percentage  Weighted_Percentage
Code   Value                                                        
_STATE 1         Alabama       7052        1.69                 1.51
       2          Alaska       2977        0.71                 0.22
       4         Arizona       8941        2.14                 2.24
       5        Arkansas       5359        1.28                 0.92
       6      California      11613        2.78                12.22

Make a list of columns that contain `float` values.

In [10]:
float_cols = []
for index, row in catalog.iterrows():
    if ' decimal ' in str(row['Value_Label']).lower():
        if index not in float_cols: 
            float_cols.append(index[0])
        continue

del catalog
print(float_cols)

['_STSTR', '_STRWT', '_RAWRAKE', '_WT2RAKE', '_CLLCPWT', '_LLCPWT2', 'HTM4', 'WTKG3', '_BMI5', '_BMI5CAT', '_RFBMI5', 'METVL11_', 'METVL21_', 'MAXVO21_', 'FC601_', 'PAFREQ1_', 'PAFREQ2_', 'STRFREQ_', 'FTJUDA2_', 'FRUTDA2_', 'GRENDA1_', 'FRNCHDA_', 'VEGEDA2_', '_FRUTSU1', '_VEGESU1']


#### Build the dataframe from the list of rows

Extract the `colspecs` from the `layout` table. It will be used to split the strings in the dataframe into multiple columns.

In [11]:
colspecs = []
for index, row in layout.iterrows():
    colspecs.append((row.Start-1, row.Length+row.Start-1))
names = layout.index.get_level_values(0).tolist()

Split the raw data into multiple columns and build a dataframe.

In [12]:
size = 5000
chunks = []

for pos in range(0, len(data_rows), size):
    rows = data_rows[pos:pos+size]
    if not rows:
        break

    file = StringIO(''.join(rows))
    file.seek(0)
    chunk = pd.read_fwf(file, names=names, colspecs=colspecs, header=None)
    chunk.fillna(pd.NA, inplace=True)
    for col in chunk.columns:
        if col in float_cols:
            chunk[col] = chunk[col].astype(pd.Float32Dtype())
        else:
            try:
                chunk[col] = chunk[col].astype(pd.Int32Dtype())
            except:
                pass

    chunks.append(chunk)

del data_rows
data = pd.concat(chunks, ignore_index=True)
data

,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENM1,...,_VEGESU1,_FRTLT1A,_VEGLT1A,_FRT16A,_VEG23A,_FRUITE1,_VEGETE1,_FLSHOT7,_PNEUMO3,_AIDTST4
0,1,1,1182019,1,18,2019,1100,2019000001,2019000001,1,...,114.0,1,1,1,1,0,0,2,1,2
1,1,1,1132019,1,13,2019,1100,2019000002,2019000002,1,...,121.0,1,1,1,1,0,0,1,1,2
2,1,1,1182019,1,18,2019,1100,2019000003,2019000003,1,...,164.0,1,1,1,1,0,0,1,2,2
3,1,1,1182019,1,18,2019,1200,2019000004,2019000004,1,...,<NA>,9,9,1,1,1,1,9,9,<NA>
4,1,1,1042019,1,4,2019,1100,2019000005,2019000005,1,...,178.0,1,1,1,1,0,0,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418263,72,9,3152020,3,15,2020,1100,2019006029,2019006029,<NA>,...,43.0,1,2,1,1,0,0,2,2,2
418264,72,9,3082020,3,8,2020,1100,2019006030,2019006030,<NA>,...,142.0,1,1,1,1,0,0,<NA>,<NA>,2
418265,72,9,3102020,3,10,2020,1100,2019006031,2019006031,<NA>,...,55.0,1,2,1,1,0,0,<NA>,<NA>,1
418266,72,9,3062020,3,6,2020,1100,2019006032,2019006032,<NA>,...,214.0,1,1,1,1,0,0,2,2,2


#### Upload the processed data to the database

In [17]:
pd.io.sql.to_sql(data,name='data', con=engine, if_exists='replace', chunksize=1000, method='multi', index=False, flavor='mysql')

418268

In [19]:
q_col_count = f'''
SELECT count(*) AS col_count 
FROM information_schema.columns 
WHERE table_name = 'data';
'''

q_row_count = f'''
SELECT count(*) as rows_count
FROM data;
'''
pd.concat([
    pd.read_sql_query(q_col_count, engine).iloc[0,:],
    pd.read_sql_query(q_row_count, engine).iloc[0,:]
])

col_count        342
rows_count    418268
Name: 0, dtype: int64